Problem statement

In this dataset, you are presented pairs of phrases (an anchor and a target phrase) and asked to rate how similar they are on a scale from 0 (not at all similar) to 1 (identical in meaning). 

Import libraries

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

Load datasets

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Read datasets

In [ ]:
train = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/train.csv")
test = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/test.csv")
submission = pd.read_csv("/kaggle/input/us-patent-phrase-to-phrase-matching/sample_submission.csv")

In [ ]:
train

In [ ]:
test

In [ ]:
submission

Analyse score

In [ ]:
sns.distplot(train.score)

In [ ]:
plt.boxplot(train.score)

Define target

In [ ]:
target = train.score

Combine train and test

In [ ]:
combi = train.drop(['score'], axis=1).append(test)
combi

Define X and y

In [ ]:
y = target
X = combi[: len(train)]
X_test = combi[len(train) :]

Match phrase similarities

In [ ]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize

simularity = []

for i in range(len(X_test)):
    
    anchor_list = word_tokenize(X_test['anchor'][i])  
    target_list = word_tokenize(X_test['target'][i]) 

    sw = stopwords.words('english')  
    l_anchor =[];l_target =[]
    
    anchor_set = {w for w in anchor_list if not w in sw}  
    target_set = {w for w in target_list if not w in sw}
    
    rvector = anchor_set.union(target_set)  
    for w in rvector: 
        if w in anchor_set: l_anchor.append(1)
        else: l_anchor.append(0) 
        if w in target_set: l_target.append(1) 
        else: l_target.append(0) 
    c = 0
    
    for i in range(len(rvector)): 
        c += l_anchor[i]*l_target[i] 
    cosine = c / float((sum(l_anchor)*sum(l_target))**0.5) 
    #print("similarity: ", cosine)
    
    simularity.append(cosine)
    
print(len(simularity))
#print(simularity)

Prepare submission

In [ ]:
submission['score'] = simularity
submission.to_csv("submission.csv", index=False)
submission